### reading seasonal forecast

In [ ]:

import xarray as xr

with xr.open_dataset('../example_data/SEAS5_202210_2m_temperature_anomaly.nc') as ds:
    ds

### reading shape file

In [ ]:
### reading shape file
import matplotlib.path as mpath
import xarray as xr
import fiona
import shapely

# Step 1: Load the shapefile vertices
shapefile_path = '../example_data/worldmap.shp'
with fiona.open(shapefile_path, "r") as shp:
    shapes = [feature for feature in shp]

### select country shape

In [ ]:
for s in shapes:
    if s.properties['CNTRY_NAME'] == 'Germany':
        shape=s

### add a small buffer to the shape

In [ ]:
shape = shapely.geometry.shape(shape['geometry']).buffer(0.2)
x, y = shape.exterior.coords.xy

In [ ]:
shape

In [ ]:
# Plot the shape using matplotlib
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.plot(x,y)
plt.show()

### select grid points based on country shape

In [ ]:
paths = []
if isinstance(shape, shapely.Polygon):
    x, y = shape.exterior.coords.xy
    coords = list(zip(x,y))
    paths.append(mpath.Path(coords))
else:    
    for s in list(shape.geoms):
        x, y = s.exterior.coords.xy
        coords = list(zip(x,y))
        paths.append(mpath.Path(coords))

lat = ds.latitude.values
lon = ds.longitude.values

# Step 5: Create a boolean mask based on the shapefile


lons,lats = np.meshgrid(lon,lat)

mask = np.full([lat.shape[0],lon.shape[0]],False)
for p in paths:
    _mask = p.contains_points(list(zip(lons.flatten(), lats.flatten())))
    _mask = _mask.reshape(lat.shape[0],lon.shape[0])
    mask = mask|_mask
    


# Step 6: Apply the mask to the dataset
masked_dataset = ds.isel(time=0).where(mask)

# Access the masked dataset's variable like this:
masked_variable = masked_dataset['t2a']

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

masked_variable.plot(ax=ax)
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

In [ ]:
mask.shape

In [ ]:
# List of coordinates
coordinates = [(0, 0), (1, 1), (2, 0), (1, -1)]

# Create the Path
path_data = mpath.Path(coordinates)
